---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# The Knapsack Problem {#sec-knapsack}

The knapsack problem is a classic optimization problem in the field of operations research. It involves selecting a subset of items from a set of items, each with a certain weight and value, to maximize the total value while staying within a weight constraint.

More formally, given a set of $n$ items, each with a weight $w_i$ and a value $v_i$, and a knapsack with a maximum weight capacity $W$, the goal is to select a subset of items such that the sum of their weights is less than or equal to $W$, and the sum of their values is maximized.

This problem is NP-hard, meaning that it is computationally difficult to find an optimal solution for large instances of the problem. Various algorithms have been developed to solve this problem, including dynamic programming, branch and bound, and heuristic methods. The knapsack problem has applications in a variety of fields, including computer science, finance, and logistics, among others.

\begin{align}
    \text{max.} &\quad \sum_{i = 1}^nv_i x_i \label{knapsack-obj} \\
    \text{s.t.} &\quad \sum_{i=1}^n w_i x_i \leq W \label{knapsack-cons1} \\
    &\quad x_i \in \{0, 1\}, \ i = 1, \cdots, n
\end{align}